# Similarity Layer
This layer takes the similarity values from the RetrievalSystem as Inputs, which should gain the model better insights into how correlated the retrieved inputs are.

In [ ]:
import torch.nn as nn

class SimilarityLayer(nn.Module):
    def __init__(self, retrieval_number: int):
        super(SimilarityLayer, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(retrieval_number, 2 * retrieval_number),
            nn.ReLU(),
            nn.Linear(2 * retrieval_number, 2 * retrieval_number),
            nn.LayerNorm(2 * retrieval_number),
            nn.ReLU()
        )

    def forward(self, x):
        return self.model(x)


# StaticFeatureLayer
This layer has all the static features from the retrieved documents as input.

In [ ]:
class StaticFeatureLayer(nn.Module):
    def __init__(self, retrieval_number: int, hidden_dim: int, static_feature_dim: int):
        super(StaticFeatureLayer, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(static_feature_dim * retrieval_number, static_feature_dim * (retrieval_number // 2)),
            nn.ReLU(),
            nn.Linear(static_feature_dim * (retrieval_number // 2), 2 * hidden_dim),
            nn.LayerNorm(2 * hidden_dim),
            nn.ReLU()
        )

    def forward(self, x):
        return self.model(x)


# HistoricalFeatureLayer
This layer contains all the hisitorical data from the retrieved companies.

In [ ]:
class HistoricalFeatureLayer(nn.Module):
    def __init__(self, retrieval_number: int, hidden_dim: int, historical_feature_dim: int):
        super(HistoricalFeatureLayer, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(historical_feature_dim * retrieval_number, historical_feature_dim * (retrieval_number // 2)),
            nn.ReLU(),
            nn.Linear(historical_feature_dim * (retrieval_number // 2), historical_feature_dim * (retrieval_number // 2)),
            nn.Dropout(0.2),
            nn.GELU(),
            nn.Linear(historical_feature_dim * (retrieval_number // 2), 4 * hidden_dim),
            nn.LayerNorm(4 * hidden_dim),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.model(x)



# IdeaLayer
This layer encodes the textual description of our idea.

In [ ]:
class IdeaLayer(nn.Module):
    def __init__(self, hidden_dim: int, bert_dim: int):
        super(IdeaLayer, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(bert_dim, 4 * hidden_dim),
            nn.ReLU(),
            nn.Linear(4 * hidden_dim, 2 * hidden_dim)
        )

    def forward(self, x):
        return self.model(x)


# IdeaStaticLayer
This layer stores gets the input of the to be predicted idea. During training there will be inputs that belong to the idea we want to predict, but during actual predictions this will empty. This should just help the model to further understand the impact of other inputs

In [ ]:
class IdeaStaticLayer(nn.Module):
    def __init__(self, static_feature_dim: int):
        super(IdeaStaticLayer, self).__init__()
        self.model = nn.Linear(static_feature_dim, 32)

    def forward(self, x):
        return self.model(x)


# IdeaHistoricalLayer
This layer gets the historical data of the to be predicted idea as inputs. This will also be filled with existing data during training and then zerod out during acutal predictions. However, very important is that newly predicted values will be added to the input vector of this layer and the oldest value will be removed (Shift of values).

In [ ]:
class IdeaHistoricalLayer(nn.Module):
    def __init__(self, historical_idea_dim: int, hidden_dim: int):
        super(IdeaHistoricalLayer, self).__init__()
        self.model = nn.Linear(historical_idea_dim, hidden_dim//2)

    def forward(self, x):
        return self.model(x)


# 1.Fusion Layer
This fusion layer combines all the inputs from our retrieved documents.

In [ ]:
class FirstFusionLayer(nn.Module):
    def __init__(self, bert_dim: int, hidden_dim: int, retrieval_number: int):
        super(FirstFusionLayer, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(bert_dim + retrieval_number + 2 * hidden_dim + 4 * hidden_dim + 2 * retrieval_number, 8 * hidden_dim),  # 384 is the fixed text embedding dimension
            nn.ReLU(),
            nn.Linear(8 * hidden_dim,  4 * hidden_dim),
            nn.LayerNorm(4 * hidden_dim),
            nn.ReLU()
        )

    def forward(self, x):
        return self.model(x)


# 2.Fusion Layer
This fusion layer combines the output from the first fusion layer and the inputs from the idea we want to predict.

In [ ]:
class SecondFusionLayer(nn.Module):
    def __init__(self, hidden_dim: int):
        super(SecondFusionLayer, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(4 * hidden_dim + 2 * hidden_dim + 32 + 64, 8 * hidden_dim),  # 384 is the fixed text embedding dimension
            nn.GELU(),
            nn.Linear(8 * hidden_dim, 7 * hidden_dim),
            nn.LayerNorm(7 * hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.GELU(),
            nn.Linear(7 * hidden_dim, 5 * hidden_dim),
            nn.Hardswish(),
            nn.Linear(5 * hidden_dim, 4 * hidden_dim),
        )

    def forward(self, x):
        return self.model(x)


# Output Layer
This is the final output layer that compromises all nodes to a single output. The overall output will then be a regressiv prediction from this layer.

In [ ]:
class OutputLayer(nn.Module):
    def __init__(self, hidden_dim: int):
        super(OutputLayer, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(2 * hidden_dim, hidden_dim),
            nn.Hardswish(),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim //2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, 1), # Final Output
        )

    def forward(self, x):
        return self.model(x)
